In [ ]:
import cv2
import numpy as np
from PIL import Image
import io
from IPython.display import display, clear_output
from jetbot import Robot, Camera
import time

# RGB_Lib는 별도의 파일에 저장되어 있어야 합니다.
# 예: 같은 디렉토리에 RGB_Lib.py 라는 이름으로 저장
from RGB_Lib import Programing_RGB

# RGB LED 객체 초기화
# Programing_RGB 클래스 내에서 I2C 버스 1 (0x1b 주소)을 사용하도록 정의되어 있음
RGB = Programing_RGB()

# 로봇 시작 시 초기 LED 상태를 설정 (카메라 준비 중)
try:
    RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 시작 시 파란색 (카메라 대기)
    print("Robot initializing with blue LED...")
except Exception as e:
    print(f"Initial RGB LED setup failed (likely I2C error): {e}")
    print("Please check RGB LED wiring and I2C address/bus settings.")


# --- [초기 변수 정의] ---
x_position_normalized = 0
y_position_normalized = 0
angle_last = 0
angle = 0

# 로봇 객체 초기화
robot = Robot()

# 카메라 객체 초기화
camera = Camera.instance(width=224, height=224, fps=10)

# 노란색 HSV 범위 정의 (환경에 맞게 조정 필수)
yellow_lower = np.array([20, 100, 100]) # H, S, V
yellow_upper = np.array([30, 255, 255])
blue_lower = np.array([100, 100, 100])
blue_upper = np.array([140, 255, 255])
red_lower = np.array([0, 100, 100])
red_upper = np.array([10, 255, 255])

# 임시 값 (Jupyter Notebook에서 위젯을 사용하지 않을 경우)
speed_gain_slider_value = 0.55
steering_gain_slider_value = 0.33
steering_dgain_slider_value = 0.12
steering_bias_slider_value = 0.0

# --- [전진 및 정지 로직 파라미터] ---
STOP_Y_THRESHOLD = 1.0 # 1.0으로 설정하면 Y 위치에 따른 정지 로직이 사실상 비활성화됩니다.
SLOW_DOWN_Y_THRESHOLD = 1.0 # 1.0으로 설정하면 Y 위치에 따른 감속 로직이 사실상 비활성화됩니다.
MIN_SPEED_VALUE = 0.5

# 라인 탐색을 위한 회전 속도
SEARCH_TURN_SPEED = 0.8 # 라인을 찾지 못했을 때 회전할 속도 (0.0 ~ 1.0)

print("Starting line tracing loop...")

try:
    while True:
        frame = camera.value # 카메라에서 최신 프레임 가져오기

        # 카메라에서 유효한 프레임이 없을 경우 처리
        if frame is None:
            try:
                RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 카메라 에러 시 빨간색 LED
            except:
                pass # I2C 오류가 나도 프로그램 멈추지 않게
            print("Warning: No frame received from camera. Stopping robot. Retrying...")
            robot.stop() # 로봇 정지
            # 모터 값 출력: 카메라 에러 시 모터 정지 상태
            print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Camera Error)")
            time.sleep(0.1) # 짧게 대기 후 다시 시도
            continue # 다음 루프에서 다시 시도

        # 이미지 크기 조정
        frame = cv2.resize(frame, (224, 224))
        size_x = frame.shape[1] # 이미지 너비
        size_y = frame.shape[0] # 이미지 높이

        # BGR 이미지를 HSV로 변환 및 마스크 생성
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_frame, yellow_lower, yellow_upper)
        contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        steering_value = 0.0
        current_speed = speed_gain_slider_value
        
        # 라인 감지 여부에 따른 모터 값 결정 로직의 시작 부분에 초기값 설정
        left_speed = 0.0
        right_speed = 0.0

        if len(contours) != 0: # 윤곽선이 하나라도 감지된 경우 (라인 발견)
            largest_contour = max(contours, key=cv2.contourArea)
            M = cv2.moments(largest_contour)

            try:
                if M["m00"] != 0:
                    cx = int(M["m10"] / M["m00"])
                    cy = int(M["m01"] / M["m00"])

                    cv2.circle(frame, (cx, cy), 5, (0, 255, 0), -1)

                    x_position_normalized = (cx / size_x) * 2 - 1
                    y_position_normalized = cy / size_y

                    # --- [전진 속도 조절 로직 (Y 위치 기반)] ---
                    # 현재 STOP_Y_THRESHOLD와 SLOW_DOWN_Y_THRESHOLD가 1.0으로 설정되어 있어
                    # 이 로직은 거의 작동하지 않을 것입니다. 필요에 따라 값을 조정하세요.
                    if y_position_normalized > STOP_Y_THRESHOLD:
                        current_speed = 0.0 # 완전 정지
                        try:
                            RGB.Set_All_RGB(0x00, 0x00, 0xFF) # 파란색 (정지 상태 표시)
                        except:
                            pass
                        print(f"Yellow line at Y={y_position_normalized:.2f} (>{STOP_Y_THRESHOLD:.2f}). Stopping.")
                    elif y_position_normalized > SLOW_DOWN_Y_THRESHOLD:
                        speed_reduction_factor = (y_position_normalized - SLOW_DOWN_Y_THRESHOLD) / (STOP_Y_THRESHOLD - SLOW_DOWN_Y_THRESHOLD)
                        current_speed = max(MIN_SPEED_VALUE, speed_gain_slider_value * (1 - speed_reduction_factor))
                        try:
                            RGB.Set_All_RGB(0x00, 0xFF, 0x00) # 초록색 (라인 감지 및 이동)
                        except:
                            pass
                        print(f"Yellow line at Y={y_position_normalized:.2f}. Slowing down to {current_speed:.2f}")
                    else: # 라인이 감지되었지만 아직 멀리 있을 때
                        current_speed = speed_gain_slider_value # 기본 속도로 전진
                        try:
                            RGB.Set_All_RGB(0x00, 0xFF, 0x00) # 초록색 (라인 감지 및 이동)
                        except:
                            pass
                        # print(f"Yellow line at Y={y_position_normalized:.2f}. Moving at full speed.")
                    # --- [전진 속도 조절 로직 끝] ---

                    # PID 제어 계산 (조향)
                    angle = x_position_normalized
                    pid = angle * steering_gain_slider_value + \
                          (angle - angle_last) * steering_dgain_slider_value
                    angle_last = angle
                    steering_value = pid + steering_bias_slider_value
                    
                    # 라인이 감지되고 움직이는 경우 모터 값 계산
                    left_speed = float(np.clip(current_speed + steering_value, 0.0, 1.0))
                    right_speed = float(np.clip(current_speed - steering_value, 0.0, 1.0))


            except Exception as e:
                robot.stop()
                try:
                    RGB.Set_All_RGB(0xFF, 0x00, 0xFF) # 보라색 (모멘트 계산 오류 상태 표시)
                except:
                    pass
                print(f"Error calculating moments for contour: {e}. Stopping robot.")
                # 모터 값 출력: 모멘트 계산 오류 시 모터 정지 상태
                print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Contour Error)")
                continue

        else: # 윤곽선(노란색 라인)이 전혀 감지되지 않을 때 (라인 미발견)
            try:
                RGB.Set_All_RGB(0xFF, 0x00, 0x00) # 빨간색 (라인 미감지 상태 표시)
            except:
                pass
            
            # 라인을 찾을 때까지 제자리에서 회전 (예: 오른쪽으로 회전)
            left_speed = SEARCH_TURN_SPEED
            right_speed = -SEARCH_TURN_SPEED # 왼쪽 바퀴 전진, 오른쪽 바퀴 후진 -> 제자리 오른쪽 회전

            print(f"No yellow line detected. Searching by turning. Motor Values: Left = {left_speed:.2f}, Right = {right_speed:.2f}")

        # 로봇 모터 값 설정
        # current_speed가 0.0이면 정지, 그렇지 않으면 계산된 left_speed, right_speed 적용
        if current_speed == 0.0:
            robot.stop()
            # 모터 값 출력: 라인 가까움으로 인한 정지 상태 (STOP_Y_THRESHOLD가 1.0이라 거의 작동 안 함)
            print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Stopped by Y-Threshold)")
        else:
            robot.left_motor.value = max(min(left_speed, 1.0), 0.0)
            robot.right_motor.value = max(min(right_speed, 1.0), 0.0)
            # 모터 값 출력: 정상 주행 또는 탐색 회전 상태
            if max(min(left_speed, 1.0), 0.0) == 0  and max(min(right_speed, 1.0), 0.0) == 0:
                robot.left_motor.value = SEARCH_TURN_SPEED
                robot.right_motor.value = -SEARCH_TURN_SPEED
            print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Moving/Searching)")


        # OpenCV 이미지를 PIL 이미지로 변환 및 Jupyter에 표시
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_byte_arr = io.BytesIO()
        image.save(img_byte_arr, format='jpeg')
        img_byte_arr = img_byte_arr.getvalue()

        clear_output(wait=True)
        display(Image.open(io.BytesIO(img_byte_arr)))

except KeyboardInterrupt:
    print("Line tracing stopped by user via KeyboardInterrupt.")
    robot.stop()
    try:
        RGB.Set_All_RGB(0x00, 0x00, 0x00) # LED 끔
    except:
        pass
    print(f"Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f} (Keyboard Interrupt)")
finally:
    if 'camera' in locals() and camera is not None:
        camera.stop()
    robot.stop()
    try:
        RGB.Set_All_RGB(0x00, 0x00, 0x00) # 최종적으로 LED 끔
    except:
        pass
    print("Program terminated.")
    print(f"Final Motor Values: Left = {robot.left_motor.value:.2f}, Right = {robot.right_motor.value:.2f}")